In [17]:
import Window_Size
import nd_lib_v3

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import numpy as np

import Post_Processing
pp = Post_Processing.PP()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
original_results = pp.read_file(fileName='1025_temporal_test`.pkl')

In [19]:
print(len(original_results))
print(len(original_results['BASE']))
print(len(original_results['BASE'][0]))

3
4
105


In [20]:
print(original_results['BASE'][0])

[{'Menu': {'windowSize': 20, 'getRawData': 'no', 'doublingData': 'no', 'stage': '1', 'rangeIndex': '0', 'avgTrials': 'no', 'avgNum': '4', 'normType': 'sk', 'turnOnMeanRemoval': 'no', 'turnOnSliceWindow': 'yes', 'sliceWindowType': 'cf', 'cfDegree': '1', 'baseOrFlip': 'base', 'turnOnPga': 'no', 'newTDT': 'clustered1', 'pcaOn': 'no', 'pcaComponents': '0.9'}, 'Accuracy_Name_linear': 0.2, 'Accuracy_Category_linear': 0.5166666666666667, 'Accuracy_Name_rbf': 0.16666666666666666, 'Accuracy_Category_rbf': 0.55, 'Accuracy_Name_lda': 0.15, 'Accuracy_Category_lda': 0.55, 'Accuracy_Name_rvm': 0.13333333333333333, 'Accuracy_Category_rvm': 0.5, 'Accuracy_Name_gnb': 0.2, 'Accuracy_Category_gnb': 0.48333333333333334, 'Accuracy_Name_linear_random': 0.11666666666666667, 'Accuracy_Category_linear_random': 0.38333333333333336, 'Accuracy_Name_AccuracyPerObject': ([0.0, 0.3, 0.3, 0.0, 0.1, 0.0], 0.11666666666666667), 'Accuracy_Category_AccuracyPerObject': ([0.26666666666666666, 0.5], 0.38333333333333336)}, {

In [21]:
def clean_data_step1(results, dataType='base', classifier='linear'):
    if dataType == 'pga':
        data = results['PGA']
    elif dataType == 'base':
        data = results['BASE']
    elif dataType == 'flip':
        data = results['FLIP']

    results_name = []
    results_cate = []
    for i in range(len(data)):
        temp_holder_name = []
        temp_holder_cate = []
        for j in range(len(data[0])):
            temp_holder_name.append(data[i][j]['Accuracy_Name_' + classifier])
            temp_holder_cate.append(data[i][j]['Accuracy_Category_' + classifier])
        results_name.append(temp_holder_name)
        results_cate.append(temp_holder_cate)
    return results_name, results_cate

In [6]:
base_data_name, base_data_cate = clean_data_step1(results=original_results, dataType='base', classifier='linear')

In [7]:
base_data_name = np.array(base_data_name)
print(base_data_name.shape)

(4, 105)


In [8]:
def clean_data_step2(results, number_of_subjects=7, number_of_time_range=15, dataType='base', classifier='linear'):
    results_name, results_category = clean_data_step1(results=results, dataType=dataType, classifier=classifier)
    ranges = number_of_time_range

    new_results_name = []
    new_results_category = []
    for stage_name, stage_cate in zip(results_name, results_category):
        temp_holder_stages_name = []
        temp_holder_stages_cate = []
        for subject in range(number_of_subjects):
            temp_holder_stages_name.append(stage_name[subject*ranges:subject*ranges+ranges])
            temp_holder_stages_cate.append(stage_cate[subject*ranges:subject*ranges+ranges])
        new_results_name.append(temp_holder_stages_name)
        new_results_category.append(temp_holder_stages_cate)

    new_results_name = np.array(new_results_name)
    new_results_category = np.array(new_results_category)

    return new_results_name, new_results_category

In [9]:
new_results_name, new_results_category = clean_data_step2(results=original_results, number_of_subjects=7, number_of_time_range=15, dataType='base', classifier='linear')

In [10]:
new_results_name.shape

(4, 7, 15)

In [11]:
for i in range(15):
    np.mean(new_results_name[0,:,i])

In [12]:
new_results_name[0]

array([[0.23333333, 0.11666667, 0.18333333, 0.33333333, 0.3       ,
        0.3       , 0.35      , 0.33333333, 0.21666667, 0.25      ,
        0.28333333, 0.16666667, 0.2       , 0.2       , 0.25      ],
       [0.12280702, 0.12280702, 0.15789474, 0.29824561, 0.33333333,
        0.35087719, 0.31578947, 0.31578947, 0.36842105, 0.22807018,
        0.1754386 , 0.12280702, 0.1754386 , 0.21052632, 0.22807018],
       [0.25925926, 0.24074074, 0.24074074, 0.38888889, 0.53703704,
        0.38888889, 0.35185185, 0.2962963 , 0.27777778, 0.24074074,
        0.22222222, 0.18518519, 0.16666667, 0.16666667, 0.16666667],
       [0.14893617, 0.14893617, 0.21276596, 0.17021277, 0.17021277,
        0.27659574, 0.21276596, 0.12765957, 0.17021277, 0.23404255,
        0.17021277, 0.08510638, 0.12765957, 0.19148936, 0.14893617],
       [0.14545455, 0.14545455, 0.18181818, 0.21818182, 0.29090909,
        0.29090909, 0.16363636, 0.18181818, 0.14545455, 0.16363636,
        0.12727273, 0.14545455, 0.12727273, 

In [13]:
def multi_Str(value, length):
    return [str(value * i) for i in range(1, length + 1)]

In [14]:
def create_graph(results, stage=0, subject=0, saving_address='C:\\Users\Sakai Lab\Desktop', dataType='base', classifier='linear'):
    new_results_object, new_results_category = clean_data_step2(results=results, dataType=dataType, classifier=classifier)

    classification_object = new_results_object[stage, subject, :]
    classification_category = new_results_category[stage, subject, :]

    x = ['0-100', '50-150', '100-200', '150-250', '200-300', '250-350', '300-400', '350-450', '400-500', '450-550', '500-600',
         '550-650', '600-700', '650-750', '700-800']

    plt.errorbar(x, classification_object, fmt='-^', label='Object')
    plt.errorbar(x, classification_category, fmt='-^', label='Category')
    plt.grid('on')

    plt.axhline(y=0.1667, color='#1f77b4', linestyle='--', label='Obj. Chance')
    plt.axhline(y=0.5, color='#ff7f0e', linestyle='--', label='Cat. Chance')

    # plt.xlabel("Time Ranges")
    plt.ylabel("Accuracy Of Subject 100"+str(subject+2))
    plt.title("Stage " + str(stage+1) +" With Different Time Range")
    plt.ylim(ymax=1, ymin=0)
    plt.xticks(rotation=90, fontsize=6)
    # plt.legend(loc='best')
    plt.legend(ncol=2)

    plt.savefig(saving_address + '\sub100' + str(subject+2) + '.png')
    plt.clf()
    return 0

In [15]:
create_graph(results=original_results)
plt.close()

In [22]:
pp.temporal_exam_auto_graph(results=original_results, saving_address='C:\\Users\Sakai Lab\Desktop\day1026', classifier='linear')
plt.close()